<a class="anchor" id="0.1"></a>
# **Prior Uncertainty Reduction tool**


This notebook contains code to reduce prior uncertainties using the seasonal water balances. 
An ensemble of long-term average *`ε≈ Qin-Qout`* is solved for unique *P*, *ET*, and ∆*TWS* combinations, and  disaggregated per season:

1. [monsoon season](#rainy)
1. [irrigation season](#irrigate)


<div><img src="wb_seasonal.png" width="800"></div>

* import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

* Prepare data
1. *Load the CSV files into data frames*

In [2]:
p= pd.read_csv(r"../data/CSVs/p.csv")
et= pd.read_csv(r"../data/CSVs/et.csv")
tws = pd.read_csv(r"../data/CSVs/tws.csv")

In [3]:
pcoldict = {'date':'date', 
            'P_MSWEP':'MSWEP', 
            'P_GPM':'GPM',  
            'P_CHIRPS':'CHIRPS',
            'P_TRMM': "TRMM"}
pdf = p.loc[:, p.columns.isin(pcoldict.keys())]
pdf.dropna(axis = 0, how = 'all', inplace = True)
pdf.rename(columns = pcoldict, inplace = True)
pdf

/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_8488/950133986.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf.dropna(axis = 0, how = 'all', inplace = True)
/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_8488/950133986.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf.rename(columns = pcoldict, inplace = True)


,date,GPM,CHIRPS,MSWEP,TRMM
0,1/1/2003,78.159860,23.889088,34.890133,67.360650
1,2/1/2003,39.440800,24.825464,25.760206,38.179386
2,3/1/2003,10.799560,12.422885,12.532451,13.632449
3,4/1/2003,3.564716,5.450367,4.658814,2.703959
4,5/1/2003,32.009235,5.893278,11.650642,9.877593
...,...,...,...,...,...
197,8/1/2019,215.041300,316.513370,280.025900,182.025380
198,9/1/2019,109.385086,145.231610,143.662580,97.593010
199,10/1/2019,25.885937,30.375492,11.661880,26.449814
200,11/1/2019,29.388199,9.044093,36.329536,32.840977


In [4]:
etcoldict = {'date':'date',
             'ET_SEBAL': "SEBAL",
             'ET_SSEBop':'SSEBop(a)',
             'ET_WAPOR':'WaPOR', 
             'ET_MODIS':'MOD16',  
             'ET_SSEBop_Lx':'SSEBop(b)',
             'ET_ALEXI': 'ALEXI',
             'CMRSET_ET':'CMRSET'}    
etdf= et.loc[:, et.columns.isin(etcoldict.keys())]
etdf.rename(columns = etcoldict, inplace = True)
etdf

/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_8488/1535432251.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etdf.rename(columns = etcoldict, inplace = True)


,date,SEBAL,SSEBop(a),WaPOR,MOD16,ALEXI,CMRSET,SSEBop(b)
0,1/1/2003,45.101433,24.614794,23.709158,5.500406,41.136086,42.842940,45.101433
1,2/1/2003,83.347390,49.569775,73.034790,36.098644,50.501266,69.870705,82.855652
2,3/1/2003,94.562730,98.840170,107.489204,36.544376,77.322210,106.398150,111.386322
3,4/1/2003,134.123280,139.636120,101.608390,1.660624,93.992170,77.954360,120.016685
4,5/1/2003,142.184100,197.485610,78.707150,0.312823,84.665130,52.800660,101.545921
...,...,...,...,...,...,...,...,...
223,8/1/2021,104.659630,154.011180,113.620330,39.794390,NaN,NaN,112.662933
224,9/1/2021,68.234390,138.919170,105.005430,60.236870,NaN,NaN,111.357689
225,10/1/2021,93.483280,84.971600,73.948160,34.100677,NaN,NaN,92.521996
226,11/1/2021,82.903760,33.120060,15.026457,8.373296,NaN,NaN,58.462437


In [5]:
gcoldict = {'date':'date',
            'mascon_cri': "MAS",
            'land_jpl':'JPL', 
            'land_gfz':'GFZ',  
            'land_csr':'CSR'}    
gdf = tws.loc[:, tws.columns.isin(gcoldict.keys())]
gdf.rename(columns = gcoldict, inplace = True)
gdf

,date,CSR,JPL,GFZ,MAS
0,4/1/2002,NaN,NaN,NaN,NaN
1,5/1/2002,-12.009,92.342,74.814,84.116
2,6/1/2002,21.844,13.666,20.278,-27.082
3,7/1/2002,21.139,13.225,19.624,-26.208
4,8/1/2002,21.844,13.666,20.278,-19.654
...,...,...,...,...,...
173,9/1/2016,-34.827,-39.016,4.212,-12.111
174,10/1/2016,-69.552,-69.590,-23.552,-100.288
175,11/1/2016,-71.870,-71.910,-24.337,-103.631
176,12/1/2016,-90.566,-85.145,-188.707,-200.606


2. merge the dfs to master df

In [6]:
wb_df = pdf.merge(gdf, on='date').merge(etdf, on='date')
wb_df

,date,GPM,CHIRPS,MSWEP,TRMM,CSR,JPL,GFZ,MAS,SEBAL,SSEBop(a),WaPOR,MOD16,ALEXI,CMRSET,SSEBop(b)
0,1/1/2003,78.159860,23.889088,34.890133,67.360650,-11.483,-67.442,-30.420,28.441,45.101433,24.614794,23.709158,5.500406,41.136086,42.842940,45.101433
1,2/1/2003,39.440800,24.825464,25.760206,38.179386,59.427,61.172,32.534,13.321,83.347390,49.569775,73.034790,36.098644,50.501266,69.870705,82.855652
2,3/1/2003,10.799560,12.422885,12.532451,13.632449,-10.148,-4.915,-10.873,-18.588,94.562730,98.840170,107.489204,36.544376,77.322210,106.398150,111.386322
3,4/1/2003,3.564716,5.450367,4.658814,2.703959,-78.983,-80.357,-54.490,-10.258,134.123280,139.636120,101.608390,1.660624,93.992170,77.954360,120.016685
4,5/1/2003,32.009235,5.893278,11.650642,9.877593,-23.655,-6.870,-123.206,38.596,142.184100,197.485610,78.707150,0.312823,84.665130,52.800660,101.545921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,9/1/2016,44.277000,78.631190,19.383257,28.986721,-34.827,-39.016,4.212,-12.111,93.672630,134.605480,117.755580,48.471855,NaN,NaN,96.593178
163,10/1/2016,20.971570,23.523449,13.980435,16.259796,-69.552,-69.590,-23.552,-100.288,79.971540,82.489620,100.738240,23.728660,NaN,NaN,77.363083
164,11/1/2016,0.193364,3.071118,0.000052,0.014380,-71.870,-71.910,-24.337,-103.631,78.625800,37.318610,49.887810,5.469684,NaN,NaN,62.016369
165,12/1/2016,1.633692,8.546742,3.650720,0.662611,-90.566,-85.145,-188.707,-200.606,63.409943,22.378365,30.121233,3.744196,NaN,NaN,32.299957


3. filter master df based on common dates

In [7]:
wb_df['date'] = pd.to_datetime(wb_df['date'])
wb_df = wb_df[(wb_df['date'].dt.year >= 2003) & (wb_df['date'].dt.year <= 2012)]
wb_df.reset_index(drop=True, inplace=True)
print(wb_df)

          date         GPM      CHIRPS       MSWEP        TRMM      CSR  \
0   2003-01-01   78.159860   23.889088   34.890133   67.360650  -11.483   
1   2003-02-01   39.440800   24.825464   25.760206   38.179386   59.427   
2   2003-03-01   10.799560   12.422885   12.532451   13.632449  -10.148   
3   2003-04-01    3.564716    5.450367    4.658814    2.703959  -78.983   
4   2003-05-01   32.009235    5.893278   11.650642    9.877593  -23.655   
..         ...         ...         ...         ...         ...      ...   
113 2012-08-01  490.914280  553.777340  455.379820  379.940900  165.046   
114 2012-09-01  158.015810  269.107000  121.261790  138.565580   99.136   
115 2012-10-01    5.339652   12.777679    5.689248    1.229766  -69.999   
116 2012-11-01    7.666080    3.595286    2.672277    1.486145  -72.332   
117 2012-12-01   15.894975    9.544070   11.193254   12.790445  -74.565   

         JPL      GFZ      MAS       SEBAL   SSEBop(a)       WaPOR      MOD16  \
0    -67.442  -30.

4. create ensemble list

In [8]:
plist = [ "MSWEP", "GPM", 'CHIRPS',"TRMM"]
etlist = [ 'SEBAL', "SSEBop(a)", 'WaPOR','MOD16','ALEXI','CMRSET',"SSEBop(b)"]
glist = ['MAS', "JPL",'GFZ','CSR']

combo_list = list(itertools.product(*[plist,etlist,glist]))
print("ensemble has {} members".format(str(len(combo_list))))

ensemble has 112 members


In [9]:
combo_list

[('MSWEP', 'SEBAL', 'MAS'),
 ('MSWEP', 'SEBAL', 'JPL'),
 ('MSWEP', 'SEBAL', 'GFZ'),
 ('MSWEP', 'SEBAL', 'CSR'),
 ('MSWEP', 'SSEBop(a)', 'MAS'),
 ('MSWEP', 'SSEBop(a)', 'JPL'),
 ('MSWEP', 'SSEBop(a)', 'GFZ'),
 ('MSWEP', 'SSEBop(a)', 'CSR'),
 ('MSWEP', 'WaPOR', 'MAS'),
 ('MSWEP', 'WaPOR', 'JPL'),
 ('MSWEP', 'WaPOR', 'GFZ'),
 ('MSWEP', 'WaPOR', 'CSR'),
 ('MSWEP', 'MOD16', 'MAS'),
 ('MSWEP', 'MOD16', 'JPL'),
 ('MSWEP', 'MOD16', 'GFZ'),
 ('MSWEP', 'MOD16', 'CSR'),
 ('MSWEP', 'ALEXI', 'MAS'),
 ('MSWEP', 'ALEXI', 'JPL'),
 ('MSWEP', 'ALEXI', 'GFZ'),
 ('MSWEP', 'ALEXI', 'CSR'),
 ('MSWEP', 'CMRSET', 'MAS'),
 ('MSWEP', 'CMRSET', 'JPL'),
 ('MSWEP', 'CMRSET', 'GFZ'),
 ('MSWEP', 'CMRSET', 'CSR'),
 ('MSWEP', 'SSEBop(b)', 'MAS'),
 ('MSWEP', 'SSEBop(b)', 'JPL'),
 ('MSWEP', 'SSEBop(b)', 'GFZ'),
 ('MSWEP', 'SSEBop(b)', 'CSR'),
 ('GPM', 'SEBAL', 'MAS'),
 ('GPM', 'SEBAL', 'JPL'),
 ('GPM', 'SEBAL', 'GFZ'),
 ('GPM', 'SEBAL', 'CSR'),
 ('GPM', 'SSEBop(a)', 'MAS'),
 ('GPM', 'SSEBop(a)', 'JPL'),
 ('GPM', 'SSEBop

# 1. monsoon season <a class="anchor" id="rainy"></a> 
[Prior Uncertainty Reduction tool](#0.1)

In [10]:
common_dates = wb_df['date'].tolist()
RO_list = []
for p, et, g in combo_list:
    RO = (wb_df[g] - wb_df[p] + wb_df[et]) 
    RO_dict = {'date': common_dates, 'p_et_g': f"{p}_{et}_{g}", 'result': RO.tolist(), 'p': wb_df[p].tolist(), 'et': wb_df[et].tolist(), 'deltaS': wb_df[g].tolist()}
    RO_list.append(pd.DataFrame(RO_dict))

RO_df = pd.concat(RO_list, ignore_index=True)

RO_df['date'] = pd.to_datetime(RO_df['date'])

print(RO_df) 

RO_df['year'] = RO_df['date'].dt.year
RO_df['month'] = RO_df['date'].dt.month

#RO = RO_df[RO_df['p_et_g'] == 'TRMM_ALEXI_JPL'][['date', 'result', 'p_et_g']]
#RO.to_csv('RO.csv')

# filter from June to September
RO_df = RO_df[(RO_df['month'] >= 6) & (RO_df['month'] <= 9)]
RO_df.drop(columns=['year', 'month'], inplace=True)
RO_df.reset_index(drop=True, inplace=True)
RO_df.set_index('date', inplace=True)

#RO_df
grp=RO_df.groupby('p_et_g').resample('Y').sum()
grp.drop('p_et_g', axis=1, inplace=True)
avg_annual_runoff = grp.groupby('p_et_g').mean().reset_index()
print(avg_annual_runoff)



            date              p_et_g      result           p          et  \
0     2003-01-01     MSWEP_SEBAL_MAS   38.652300   34.890133   45.101433   
1     2003-02-01     MSWEP_SEBAL_MAS   70.908184   25.760206   83.347390   
2     2003-03-01     MSWEP_SEBAL_MAS   63.442279   12.532451   94.562730   
3     2003-04-01     MSWEP_SEBAL_MAS  119.206466    4.658814  134.123280   
4     2003-05-01     MSWEP_SEBAL_MAS  169.129458   11.650642  142.184100   
...          ...                 ...         ...         ...         ...   
13211 2012-08-01  TRMM_SSEBop(b)_CSR -149.751300  379.940900   65.143600   
13212 2012-09-01  TRMM_SSEBop(b)_CSR   42.744286  138.565580   82.173866   
13213 2012-10-01  TRMM_SSEBop(b)_CSR   11.100000    1.229766   82.328766   
13214 2012-11-01  TRMM_SSEBop(b)_CSR  -30.581416    1.486145   43.236729   
13215 2012-12-01  TRMM_SSEBop(b)_CSR  -30.650462   12.790445   56.704983   

        deltaS  
0       28.441  
1       13.321  
2      -18.588  
3      -10.258  
4 

# 2. irrigation season <a class="anchor" id="irrigate"></a> 
[Prior Uncertainty Reduction tool](#0.1)

In [12]:
imports_list = []  
common_dates = wb_df['date'].tolist()
for p, et, g in combo_list:
    inflows=wb_df[g] - wb_df[p]+ wb_df[et]
    inflows_combos = {'date': common_dates, 'p_et_g': f"{p}_{et}_{g}", 'result': inflows.tolist(), 'p':wb_df[p].tolist(),'et':wb_df[et].tolist(),'deltaS':wb_df[g].tolist()}
    imports_list.append(pd.DataFrame(inflows_combos))


IMP_df = pd.concat(imports_list, ignore_index=True)

IMP_df['date'] = pd.to_datetime(IMP_df['date'])

print(IMP_df) 

IMP_df['year'] = IMP_df['date'].dt.year
IMP_df['month'] = IMP_df['date'].dt.month

#wimports = IMP_df[IMP_df['p_et_g'] == 'TRMM_SEBAL_JPL'][['date', 'result', 'p_et_g']]
#wimports.to_csv('wimports.csv')

#filter df for months outside monsoon
rabi_df = IMP_df[((IMP_df['month'] >= 1) & (IMP_df['month'] <= 5)) | ((IMP_df['month'] >= 10) & (IMP_df['month'] <= 12))]
rabi_df.drop(columns=['year', 'month'], inplace=True)
rabi_df.reset_index(drop=True, inplace=True)
rabi_df['date'] = pd.to_datetime(rabi_df['date'])
rabi_df.set_index('date', inplace=True)

#RO_df
grp_rabi=rabi_df.groupby('p_et_g').resample('Y').sum()
grp_rabi.drop('p_et_g', axis=1, inplace=True)

# print epsilon for irrigation season
avg_annual_inflow = grp_rabi.groupby('p_et_g').mean().reset_index()
avg_annual_inflow



            date              p_et_g      result           p          et  \
0     2003-01-01     MSWEP_SEBAL_MAS   38.652300   34.890133   45.101433   
1     2003-02-01     MSWEP_SEBAL_MAS   70.908184   25.760206   83.347390   
2     2003-03-01     MSWEP_SEBAL_MAS   63.442279   12.532451   94.562730   
3     2003-04-01     MSWEP_SEBAL_MAS  119.206466    4.658814  134.123280   
4     2003-05-01     MSWEP_SEBAL_MAS  169.129458   11.650642  142.184100   
...          ...                 ...         ...         ...         ...   
13211 2012-08-01  TRMM_SSEBop(b)_CSR -149.751300  379.940900   65.143600   
13212 2012-09-01  TRMM_SSEBop(b)_CSR   42.744286  138.565580   82.173866   
13213 2012-10-01  TRMM_SSEBop(b)_CSR   11.100000    1.229766   82.328766   
13214 2012-11-01  TRMM_SSEBop(b)_CSR  -30.581416    1.486145   43.236729   
13215 2012-12-01  TRMM_SSEBop(b)_CSR  -30.650462   12.790445   56.704983   

        deltaS  
0       28.441  
1       13.321  
2      -18.588  
3      -10.258  
4 

/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_8488/354217831.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rabi_df.drop(columns=['year', 'month'], inplace=True)
/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_8488/354217831.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rabi_df['date'] = pd.to_datetime(rabi_df['date'])


,p_et_g,result,p,et,deltaS
0,CHIRPS_ALEXI_CSR,59.669771,124.858597,494.815569,-310.2872
1,CHIRPS_ALEXI_GFZ,74.820271,124.858597,494.815569,-295.1367
2,CHIRPS_ALEXI_JPL,82.250571,124.858597,494.815569,-287.7064
3,CHIRPS_ALEXI_MAS,21.517271,124.858597,494.815569,-348.4397
4,CHIRPS_CMRSET_CSR,109.044234,124.858597,544.190031,-310.2872
...,...,...,...,...,...
107,TRMM_SSEBop(b)_MAS,58.726433,165.198790,572.364922,-348.4397
108,TRMM_WaPOR_CSR,39.549328,165.198790,515.035318,-310.2872
109,TRMM_WaPOR_GFZ,54.699828,165.198790,515.035318,-295.1367
110,TRMM_WaPOR_JPL,62.130128,165.198790,515.035318,-287.7064






## Thanks for viewing!


